In [137]:
import_statements(ceil)

from sage.functions.other import ceil


In [2]:
from itertools import combinations_with_replacement

from ehrhart_quasi_polynomial import get_period, get_gcd, create_polytope_from_matrix
from ehrhart_quasi_polynomial import PiecewiseEhrhartQuasiPolynomial as PEQP
from ehrhart_quasi_polynomial import *

from sage.matrix.constructor import Matrix as create_matrix

In [3]:
ehr = lambda A, b: ehrhart_quasi_polynomial(create_polytope_from_matrix(A, b).Vrepresentation())

In [4]:
num_int_points = lambda A, b: len(create_polytope_from_matrix(A, b).integral_points())

In [133]:
# seems to work great if the cone is generated by a single ray
def _displacement_factor(self, lifted_off_set, cone, proj_sum, non_zero_indexes):
    proj_off = self._projection(lifted_off_set)
    disp_fac = ceil( -min( proj_off[k]/proj_sum[k] for k in non_zero_indexes) ) + 3
    return disp_fac

def _compute_piecewise(self):
        num_variables = self._A.nrows() # = self._amb_dim
        max_degree = self._A.ncols()
        needed_points = factorial(num_variables + max_degree)//(
            factorial(num_variables)*factorial(max_degree) )

        R = PolynomialRing(QQ, "x", num_variables)

        for idx, cone_dict in enumerate(self._cone_dicts):
            scaled_rays = cone_dict["scaled_rays"] + self._lin_vectors
            cone_points = self._generate_cone_points(scaled_rays, needed_points)

            ray_sum = free_module_element(sum(scaled_rays))
            proj_sum = self._projection(ray_sum)
            non_zero_indexes = [k for k, val in enumerate(proj_sum) if val != 0]

            polynomials = {}
            cone = cone_dict["cone"]
            for unlifted in cone_dict["quotient"]:
                lifted = unlifted.lift()
                print(lifted)

                disp_fac = self._displacement_factor(lifted, cone, proj_sum, non_zero_indexes)
                starting_point = lifted + disp_fac*ray_sum
                print(self._projection(starting_point) in cone)
                off_cone_points = [p + starting_point for p in cone_points]

                num_integral_points = []
                for point in off_cone_points:
                    polytope = self._create_polytope_from_matrix(point)
                    num_integral_points.append(len(polytope.integral_points()))

                interpolated = R.interpolation(max_degree, off_cone_points,
                                               num_integral_points)
                polynomials[unlifted] = interpolated
            self._cone_dicts[idx]["polynomials"] = polynomials


In [111]:
set()
A = Matrix([[-1, 0], [0, -1], [1, 1]])
p = PEQP(A) 
for b in combinations_with_replacement(range(-12, 13), 3):
    expected = num_int_points(A, b)
    actual = p(b)
    if actual != expected:
        print(b, expected, actual)
print("all points were tested")

[{'rays': [(1, 1, 1)], 'cone': 1-d cone in 3-d lattice N, 'scaled_rays': [(1, 1, 1)], 'quotient': Finitely generated module V/W over Integer Ring with invariants (3)}]
(0, 0, 0)
True
(-1, 0, 0)
True
(-2, 0, 0)
True
all points were tested


In [112]:
p._cone_dicts[0]["polynomials"]

{(0): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (1): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (2): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1}

In [ ]:
set()
B = Matrix([[-1, 0], [0, -1], [2, 2]])
q = PEQP(B)
for b in combinations_with_replacement(range(-10, 11), 3):
    expected = num_int_points(B, b)
    actual = q(b)
    if actual != expected:
        print(b, expected, q._cone_dicts[0]["quotient"](b), actual)
print("all points were tested")

In [ ]:
q._cone_dicts[0]["polynomials"]

In [134]:
set()
C = Matrix([[-1, 0], [0, -1], [1, 1], [0, 1]])
r = PEQP(C)

[{'rays': [(-1, 2, -1, 3), (1, 3, 1, 2)], 'cone': 2-d cone in 4-d lattice N, 'scaled_rays': [(-1, 2, -1, 3), (1, 3, 1, 2)], 'quotient': Finitely generated module V/W over Integer Ring with invariants (5, 5)}, {'rays': [(1, 3, 1, 2), (2, 1, 2, -1)], 'cone': 2-d cone in 4-d lattice N, 'scaled_rays': [(1, 3, 1, 2), (2, 1, 2, -1)], 'quotient': Finitely generated module V/W over Integer Ring with invariants (5, 5)}]
(0, 0, 0, 0)
True
(0, -1, 1, -2)
True
(0, -2, 2, -4)
True
(0, -3, 3, -6)
True
(0, -4, 4, -8)
True
(-3, 4, -3, 8)
True
(-3, 3, -2, 6)
True
(-3, 2, -1, 4)
True
(-3, 1, 0, 2)
True
(-3, 0, 1, 0)
True
(-6, 8, -6, 16)
True
(-6, 7, -5, 14)
True
(-6, 6, -4, 12)
True
(-6, 5, -3, 10)
True
(-6, 4, -2, 8)
True
(-9, 12, -9, 24)
False
(-9, 11, -8, 22)
False
(-9, 10, -7, 20)
False
(-9, 9, -6, 18)
True
(-9, 8, -5, 16)
True
(-12, 16, -12, 32)
False
(-12, 15, -11, 30)
False
(-12, 14, -10, 28)
False
(-12, 13, -9, 26)
False
(-12, 12, -8, 24)
False
(0, 0, 0, 0)
True
(-1, -1, 0, -1)
True
(-2, -2, 0, 

In [135]:
r._cone_dicts[0]["polynomials"]

{(0, 0): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (0, 1): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (0, 2): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (0, 3): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (0, 4): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (1, 0): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (1, 1): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (1, 2): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (1, 3): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (1, 4): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x0*x2 + x1*x2 + 1/2*x2^2 + 3/2*x0 + 3/2*x1 + 3/2*x2 + 1,
 (2, 0): 1/2*x0^2 + x0*x1 + 1/2*x1^2 + x

In [136]:
r._cone_dicts[1]["polynomials"]

{(0, 0): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2 + x0 + 3/2*x1 + x2 + 1/2*x3 + 1,
 (0, 1): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2 + x0 + 3/2*x1 + x2 + 1/2*x3 + 1,
 (0, 2): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2 + x0 + 3/2*x1 + x2 + 1/2*x3 + 1,
 (0, 3): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2 + x0 + 3/2*x1 + x2 + 1/2*x3 + 1,
 (0, 4): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2 + x0 + 3/2*x1 + x2 + 1/2*x3 + 1,
 (1, 0): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2 + x0 + 3/2*x1 + x2 + 1/2*x3 + 1,
 (1, 1): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2 + x0 + 3/2*x1 + x2 + 1/2*x3 + 1,
 (1, 2): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2 + x0 + 3/2*x1 + x2 + 1/2*x3 + 1,
 (1, 3): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2 + x0 + 3/2*x1 + x2 + 1/2*x3 + 1,
 (1, 4): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2 + x0 + 3/2*x1 + x2 + 1/2*x3 + 1,
 (2, 0): x0*x1 + 1/2*x1^2 + x1*x2 + x0*x3 + x2*x3 - 1/2*x3^2

In [60]:
for c in combinations_with_replacement(range(-10, 11), 4):
    expected = num_int_points(C, c)
    actual = r(c)
    if actual != expected:
        print(c, expected, actual)
print("all points were tested")

all points were tested


In [ ]:
set()
D = Matrix([[-1, 0], [0, -1], [1, 2], [0, 1]])
s = PEQP(D)

In [ ]:
s._cone_dicts[0]["polynomials"]

In [88]:
for d in combinations_with_replacement(range(-9, 10), 4):
    expected = num_int_points(D, d)
    actual = s(d)
    if actual != expected:
        print(d, expected, actual)
print("all points were tested")

all points were tested


In [63]:
set()
D = Matrix([[-1, 0], [0, -1], [2, 2], [0, 1]])
s = PEQP(D)

(0, 0, 0, 0)
(0, 0, 0, 0)
(-17, 17, -7, 34)
(-17, 101, -7, 118)
(-34, 34, -14, 68)
(-34, 160, -14, 194)
(-51, 51, -21, 102)
(-51, 233, -21, 284)
(-68, 68, -28, 136)
(-68, 292, -28, 360)
(-85, 85, -35, 170)
(-85, 379, -35, 464)
(-102, 102, -42, 204)
(-102, 438, -42, 540)
(-119, 119, -49, 238)
(-119, 497, -49, 616)
(-136, 136, -56, 272)
(-136, 570, -56, 706)
(-153, 153, -63, 306)
(-153, 643, -63, 796)
(-170, 170, -70, 340)
(-170, 716, -70, 886)
(-187, 187, -77, 374)
(-187, 775, -77, 962)
(-204, 204, -84, 408)
(-204, 834, -84, 1038)
(-221, 221, -91, 442)
(-221, 907, -91, 1128)
(-238, 238, -98, 476)
(-238, 980, -98, 1218)
(-255, 255, -105, 510)
(-255, 1053, -105, 1308)
(-272, 272, -112, 544)
(-272, 1112, -112, 1384)
(-289, 289, -119, 578)
(-289, 1185, -119, 1474)
(-306, 306, -126, 612)
(-306, 1258, -126, 1564)
(-323, 323, -133, 646)
(-323, 1317, -133, 1640)
(-340, 340, -140, 680)
(-340, 1390, -140, 1730)
(-357, 357, -147, 714)
(-357, 1449, -147, 1806)
(-374, 374, -154, 748)
(-374, 1536, -1

KeyboardInterrupt: 

In [98]:
E = Matrix([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [1, 1, 1]])
t = PEQP(E)

[{'rays': [(1, 1, 1, 1)], 'cone': 1-d cone in 4-d lattice N, 'scaled_rays': [(1, 1, 1, 1)], 'quotient': Finitely generated module V/W over Integer Ring with invariants (4)}]
(0, 0, 0, 0)
True
(-1, -1, 0, 1)
True
(-2, -2, 0, 2)
True
(-3, -3, 0, 3)
True


In [73]:
for e in combinations_with_replacement(range(-10, 11), 4):
    expected = num_int_points(E, e)
    actual = t(e)
    if actual != expected:
        print(e, expected, actual)
print("all points were tested")

all points were tested


In [99]:
t._cone_dicts[0]["polynomials"]

{(0): 1/6*x0^3 + 1/2*x0^2*x1 + 1/2*x0*x1^2 + 1/6*x1^3 + 1/2*x0^2*x2 + x0*x1*x2 + 1/2*x1^2*x2 + 1/2*x0*x2^2 + 1/2*x1*x2^2 + 1/6*x2^3 + 1/2*x0^2*x3 + x0*x1*x3 + 1/2*x1^2*x3 + x0*x2*x3 + x1*x2*x3 + 1/2*x2^2*x3 + 1/2*x0*x3^2 + 1/2*x1*x3^2 + 1/2*x2*x3^2 + 1/6*x3^3 + x0^2 + 2*x0*x1 + x1^2 + 2*x0*x2 + 2*x1*x2 + x2^2 + 2*x0*x3 + 2*x1*x3 + 2*x2*x3 + x3^2 + 11/6*x0 + 11/6*x1 + 11/6*x2 + 11/6*x3 + 1,
 (1): 1/6*x0^3 + 1/2*x0^2*x1 + 1/2*x0*x1^2 + 1/6*x1^3 + 1/2*x0^2*x2 + x0*x1*x2 + 1/2*x1^2*x2 + 1/2*x0*x2^2 + 1/2*x1*x2^2 + 1/6*x2^3 + 1/2*x0^2*x3 + x0*x1*x3 + 1/2*x1^2*x3 + x0*x2*x3 + x1*x2*x3 + 1/2*x2^2*x3 + 1/2*x0*x3^2 + 1/2*x1*x3^2 + 1/2*x2*x3^2 + 1/6*x3^3 + x0^2 + 2*x0*x1 + x1^2 + 2*x0*x2 + 2*x1*x2 + x2^2 + 2*x0*x3 + 2*x1*x3 + 2*x2*x3 + x3^2 + 11/6*x0 + 11/6*x1 + 11/6*x2 + 11/6*x3 + 1,
 (2): 1/6*x0^3 + 1/2*x0^2*x1 + 1/2*x0*x1^2 + 1/6*x1^3 + 1/2*x0^2*x2 + x0*x1*x2 + 1/2*x1^2*x2 + 1/2*x0*x2^2 + 1/2*x1*x2^2 + 1/6*x2^3 + 1/2*x0^2*x3 + x0*x1*x3 + 1/2*x1^2*x3 + x0*x2*x3 + x1*x2*x3 + 1/2*x2^2*x3 + 1/2*x

In [109]:
set()
F = Matrix([[-1, 0], [0, -1], [2, 3]])
u = PEQP(F)

[{'rays': [(2, 3, 1)], 'cone': 1-d cone in 3-d lattice N, 'scaled_rays': [(6, 9, 3)], 'quotient': Finitely generated module V/W over Integer Ring with invariants (42)}]
(0, 0, 0)
True
(-1, -1, 0)
True
(-2, -2, 0)
True
(-3, -3, 0)
True
(-4, -4, 0)
True
(-5, -5, 0)
True
(-6, -6, 0)
True
(-7, -7, 0)
True
(-8, -8, 0)
True
(-9, -9, 0)
True
(-10, -10, 0)
True
(-11, -11, 0)
True
(-12, -12, 0)
True
(-13, -13, 0)
True
(-14, -14, 0)
True
(-15, -15, 0)
True
(-16, -16, 0)
True
(-17, -17, 0)
True
(-18, -18, 0)
True
(-19, -19, 0)
True
(-20, -20, 0)
True
(-21, -21, 0)
True
(-22, -22, 0)
True
(-23, -23, 0)
True
(-24, -24, 0)
True
(-25, -25, 0)
True
(-26, -26, 0)
True
(-27, -27, 0)
True
(-28, -28, 0)
True
(-29, -29, 0)
True
(-30, -30, 0)
True
(-31, -31, 0)
True
(-32, -32, 0)
True
(-33, -33, 0)
True
(-34, -34, 0)
True
(-35, -35, 0)
True
(-36, -36, 0)
True
(-37, -37, 0)
True
(-38, -38, 0)
True
(-39, -39, 0)
True
(-40, -40, 0)
True
(-41, -41, 0)
True


In [110]:
u._cone_dicts[0]["polynomials"]

{(0): 1/3*x0^2 + x0*x1 + 3/4*x1^2 + 1/3*x0*x2 + 1/2*x1*x2 + 1/12*x2^2 + x0 + 3/2*x1 + 1/2*x2 + 1,
 (1): 1/3*x0^2 + x0*x1 + 3/4*x1^2 + 1/3*x0*x2 + 1/2*x1*x2 + 1/12*x2^2 + x0 + 3/2*x1 + 1/2*x2 + 5/12,
 (2): 1/3*x0^2 + x0*x1 + 3/4*x1^2 + 1/3*x0*x2 + 1/2*x1*x2 + 1/12*x2^2 + x0 + 3/2*x1 + 1/2*x2 + 2/3,
 (3): 1/3*x0^2 + x0*x1 + 3/4*x1^2 + 1/3*x0*x2 + 1/2*x1*x2 + 1/12*x2^2 + x0 + 3/2*x1 + 1/2*x2 + 3/4,
 (4): 1/3*x0^2 + x0*x1 + 3/4*x1^2 + 1/3*x0*x2 + 1/2*x1*x2 + 1/12*x2^2 + x0 + 3/2*x1 + 1/2*x2 + 2/3,
 (5): 1/3*x0^2 + x0*x1 + 3/4*x1^2 + 1/3*x0*x2 + 1/2*x1*x2 + 1/12*x2^2 + x0 + 3/2*x1 + 1/2*x2 + 5/12,
 (6): 1/3*x0^2 + x0*x1 + 3/4*x1^2 + 1/3*x0*x2 + 1/2*x1*x2 + 1/12*x2^2 + x0 + 3/2*x1 + 1/2*x2 + 1,
 (7): 1/3*x0^2 + x0*x1 + 3/4*x1^2 + 1/3*x0*x2 + 1/2*x1*x2 + 1/12*x2^2 + x0 + 3/2*x1 + 1/2*x2 + 5/12,
 (8): 1/3*x0^2 + x0*x1 + 3/4*x1^2 + 1/3*x0*x2 + 1/2*x1*x2 + 1/12*x2^2 + x0 + 3/2*x1 + 1/2*x2 + 2/3,
 (9): 1/3*x0^2 + x0*x1 + 3/4*x1^2 + 1/3*x0*x2 + 1/2*x1*x2 + 1/12*x2^2 + x0 + 3/2*x1 + 1/2*x2 + 3/4,
 

In [104]:
for f in combinations_with_replacement(range(-12, 13), 3):
    expected = num_int_points(F, f)
    actual = u(f)
    if actual != expected:
        print(f, expected, actual)
print("all points were tested")

all points were tested
